In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [21]:
precipitation = pd.read_csv('./Dataset/Rainfall/Processed_Rainfall/precipitation_25.csv')
waterlevel = pd.read_csv('./Dataset/Water Level/Processed_WaterLevel/WL_25.csv')

In [22]:
print(precipitation.head())
print(waterlevel.head())

         date  precipitation  day  month  year
0  2025-01-01            0.0    1      1  2025
1  2025-01-02            0.0    2      1  2025
2  2025-01-03            0.0    3      1  2025
3  2025-01-04            0.0    4      1  2025
4  2025-01-05            0.0    5      1  2025
         date  water_level  day  month  year
0  2025-01-01        8.282    1      1  2025
1  2025-01-02        8.282    2      1  2025
2  2025-01-03        8.282    3      1  2025
3  2025-01-04        8.282    4      1  2025
4  2025-01-05        8.282    5      1  2025


In [23]:
merged_df = waterlevel.merge(precipitation, on=['date', 'day', 'month', 'year'])
merged_df.head()

,date,water_level,day,month,year,precipitation
0,2025-01-01,8.282,1,1,2025,0.0
1,2025-01-02,8.282,2,1,2025,0.0
2,2025-01-03,8.282,3,1,2025,0.0
3,2025-01-04,8.282,4,1,2025,0.0
4,2025-01-05,8.282,5,1,2025,0.0


In [56]:
# merged_df = pd.read_csv('./Dataset/Complete Data/final_merge.csv', index_col=0)

In [57]:
#calculating cumulative precipitation 
merged_df['precipitation_7days'] = merged_df['precipitation'].rolling(window=7, min_periods=1).sum()
merged_df['precipitation_15days'] = merged_df['precipitation'].rolling(window=15, min_periods=1).sum()
merged_df['precipitation_30days'] = merged_df['precipitation'].rolling(window=30, min_periods=1).sum()

#calculating previous days water level and precipitation
merged_df['water_level_t-1'] = merged_df['water_level'].shift(periods=1)
merged_df['water_level_t-2'] = merged_df['water_level'].shift(periods=2)
merged_df['precipitation_t-1'] = merged_df['precipitation'].shift(periods=1)
merged_df['precipitation_t-2'] = merged_df['precipitation'].shift(periods=2)
merged_df['precipitation_t-3'] = merged_df['precipitation'].shift(periods=3)

#calculating next days water level and precipitation
merged_df['target_water_level'] = merged_df['water_level'].shift(periods=-1)
merged_df['precipitation_t+1'] = merged_df['precipitation'].shift(periods=-1)
merged_df['precipitation_t+2'] = merged_df['precipitation'].shift(periods=-2)
merged_df['precipitation_t+3'] = merged_df['precipitation'].shift(periods=-3)
merged_df['precipitation_t+4'] = merged_df['precipitation'].shift(periods=-4)

#handling NAN values
merged_df['precipitation_t-1'] = merged_df['precipitation_t-1'].bfill()
merged_df['precipitation_t-2'] = merged_df['precipitation_t-2'].bfill()
merged_df['precipitation_t-3'] = merged_df['precipitation_t-3'].bfill()
merged_df['water_level_t-1'] = merged_df['water_level_t-1'].bfill()
merged_df['water_level_t-2'] = merged_df['water_level_t-2'].bfill()
merged_df['precipitation_t+1'] = merged_df['precipitation_t+1'].ffill()
merged_df['precipitation_t+2'] = merged_df['precipitation_t+2'].ffill()
merged_df['precipitation_t+3'] = merged_df['precipitation_t+3'].ffill()
merged_df['precipitation_t+4'] = merged_df['precipitation_t+4'].ffill()
merged_df['target_water_level'] = merged_df['target_water_level'].ffill()

merged_df.dropna(inplace=True)


In [58]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3282 entries, 0 to 3281
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  3282 non-null   object 
 1   water_level           3282 non-null   float64
 2   precipitation         3282 non-null   float64
 3   flood_occurred        3282 non-null   bool   
 4   precipitation_7days   3282 non-null   float64
 5   precipitation_15days  3282 non-null   float64
 6   precipitation_30days  3282 non-null   float64
 7   water_level_t-1       3282 non-null   float64
 8   water_level_t-2       3282 non-null   float64
 9   precipitation_t-1     3282 non-null   float64
 10  precipitation_t-2     3282 non-null   float64
 11  precipitation_t-3     3282 non-null   float64
 12  target_water_level    3282 non-null   float64
 13  precipitation_t+1     3282 non-null   float64
 14  precipitation_t+2     3282 non-null   float64
 15  precipitation_t+3     3282

In [ ]:
# merged_df['date'] = pd.to_datetime(merged_df['date'])
# merged_df['day'] = merged_df['date'].dt.day
# merged_df['month'] = merged_df['date'].dt.month
# merged_df['year'] = merged_df['date'].dt.year

In [61]:
merged_df.to_csv('./Dataset/Complete Data/merged_data_16-24.csv', index=False)